<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/diksha/analyse_CH_mobility_whitepop_time_series_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [1]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

time: 1.94 ms (started: 2021-04-16 04:45:08 +00:00)


In [2]:
# Install required dependancies
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 69kB/s 
     |████████████████████████████████| 204kB 38.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=7dcbb6badba801a7cc3d6df697cb628362529f58cb04fb0246f74a8354685c71
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [3]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 92.9 ms (started: 2021-04-16 04:46:00 +00:00)


In [4]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 283 ms (started: 2021-04-16 04:46:01 +00:00)


In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 4.93 s (started: 2021-04-16 04:46:01 +00:00)


In [6]:
#sc.stop()

time: 1.45 ms (started: 2021-04-16 04:46:06 +00:00)


In [23]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 4.79 ms (started: 2021-04-16 04:56:25 +00:00)


In [24]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ch'

time: 4 ms (started: 2021-04-16 04:56:26 +00:00)


In [25]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [26]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [27]:
# # Take small sample of data to experiment with
# df_soc = df_soc.limit(100)
# df_soc.show()

time: 1.07 ms (started: 2021-04-16 04:56:35 +00:00)


In [28]:
df_soc.createOrReplaceTempView('clean_ch')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, completely_home_device_count, part_time_work_behavior_devices, full_time_work_behavior_devices FROM clean_ch WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|170312304002|2020-02-07T00:00:...|          76|                          22|                              9|                              7|
|170313005002|2020-02-07T00:00:...|          72|                          27|                             12|                              9|
|170314208001|2020-02-07T00:00:...|         127|                          33|                             11|                             15|
|170317608034|2020-02-07T00:00:...|         114|                          37|                             11|                             10|
|17031

In [29]:
# View schema
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)

time: 7.38 ms (started: 2021-04-16 04:56:35 +00:00)


In [30]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
|170312304002|2020-02-07T00:00:...|          76|                          22|                              9|                              7|        28.947368421052634|       11.842105263157894|        9.210526315789473|
|170313005002|2020-02-07T00:00:...|          72|                          27|                             12|       

In [31]:
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- completely_home_percentage: double (nullable = true)
 |-- part_time_work_percentage: double (nullable = true)
 |-- full_time_work_percentage: double (nullable = true)

time: 5.86 ms (started: 2021-04-16 04:56:36 +00:00)


In [32]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 93.9 ms (started: 2021-04-16 04:56:36 +00:00)




---


**Demographic Data**






---



In [33]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'ch.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+-------------------+
|         cbg|pop_total|poverty_percentage|     perc_whiteonly|
+------------+---------+------------------+-------------------+
|170312909001|      935|  55.8288770053476|                0.0|
|170312909002|     1129|45.261293179805136|                0.0|
|170312909003|      765|  39.7078353253652|                0.0|
|170312909004|     1075|30.325581395348838|                0.0|
|170318387002|     1016|31.003937007874015| 13.484251968503939|
|170313102001|      835|16.766467065868262|  78.20359281437126|
|170313102002|      686| 16.61807580174927| 50.583090379008745|
|170314005002|      807|40.644361833952914|                0.0|
|170314804006|      710| 1.971830985915493|  1.267605633802817|
|170314804007|      650|               0.0|                0.0|
|170314805001|     1141|  9.55302366345311|  4.206836108676599|
|170312608002|     1100| 55.45454545454545| 2.8181818181818183|
|170312609001|      836| 44.856459330143

In [34]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.perc_whiteonly from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|    perc_whiteonly|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|170312304002|2020-02-07T00:00:...|          76|                          22|                              9|                              7|        28.947368421052634|       11.842105263157894|        9.210526315789473| 51.01351351351351|
|170313005002|2020-02-07T00:00:...|     

In [35]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE perc_whiteonly > 80')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_white_majority')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE perc_whiteonly < 20')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_white_minority')



grouped_df_mob_demo_combined = spark.sql("""

SELECT wh.date,wh.completely_home_percentage as high_comp_home_perc,
wl.completely_home_percentage as low_comp_home_perc  
FROM mob_demo_white_majority wh INNER JOIN mob_demo_white_minority wl ON wh.date = wl.date

""")


time: 172 ms (started: 2021-04-16 04:56:37 +00:00)


In [36]:
#grouped_df_mob_demo_combined.show()


time: 941 µs (started: 2021-04-16 04:56:37 +00:00)


In [37]:
grouped_df_mob_demo_combined_pandas = grouped_df_mob_demo_combined.toPandas()
grouped_df_mob_demo_combined_pandas

,date,high_comp_home_perc,low_comp_home_perc
0,2020-04-10T00:00:00-05:00,44.424543,41.380686
1,2020-05-12T00:00:00-05:00,40.724944,40.059169
2,2020-06-14T00:00:00-05:00,33.460101,37.162803
3,2020-07-03T00:00:00-05:00,27.520350,30.974934
4,2020-10-21T00:00:00-05:00,30.660111,35.433094
...,...,...,...
361,2020-01-25T00:00:00-06:00,23.702325,30.522261
362,2020-09-15T00:00:00-05:00,30.461303,35.060581
363,2020-10-25T00:00:00-05:00,34.472705,36.841690
364,2020-10-08T00:00:00-05:00,28.221108,32.901128


time: 3min 43s (started: 2021-04-16 04:56:37 +00:00)


---
**Graph** 

---

In [38]:
base = alt.Chart(grouped_df_mob_demo_combined_pandas.reset_index()).encode(x='date')

alt.layer(
    base.mark_line(color='blue').encode(y='high_comp_home_perc'),
    
    base.mark_line(color='red').encode(y='low_comp_home_perc')

)

alt.LayerChart(...)

time: 67.8 ms (started: 2021-04-16 05:00:21 +00:00)
